# Interogarile din baza de date clinicaStomatologica ce contine 8 tabele
#### *Proiect realizat de Bargan Diana-Georgiana, Irimia Emanuela-Nicoletta, Vasilache Vlad-Alexandru, echipa DM1105*
#### Tabelele utilizate sunt: pacienti, medici, programari, tratamente, consultatii, istoricMedical, facturi, plati.

In [1]:
from pyspark.shell import sqlContext
from pyspark.sql import SparkSession

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 18:23:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.0
      /_/

Using Python version 3.10.12 (main, Nov 20 2023 15:14:05)
Spark context Web UI available at http://master:4040
Spark context available as 'sc' (master = spark://master:7077, app id = app-20240516182352-0041).
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = (
    SparkSession.builder.appName("Python Spark SQL basic example")
    .config("spark.executor.memory", "1G")
    .config("spark.executor.cores", "2")
    .config("master", "spark://master:7077")
    .getOrCreate()
)

24/05/16 18:23:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
pacienti= spark.read.csv("/proiectdm1105/pacienti.csv", inferSchema=True, header=True)
medici= spark.read.csv("/proiectdm1105/medici.csv", inferSchema=True, header=True)
programari= spark.read.csv("/proiectdm1105/programari.csv", inferSchema=True, header=True)
consultatii= spark.read.csv("/proiectdm1105/consultatii.csv", inferSchema=True, header=True)
istoricMedical= spark.read.csv("/proiectdm1105/istoricMedical.csv", inferSchema=True, header=True)
tratamente= spark.read.csv("/proiectdm1105/tratamente.csv", inferSchema=True, header=True)
facturi= spark.read.csv("/proiectdm1105/facturi.csv", inferSchema=True, header=True)
plati= spark.read.csv("/proiectdm1105/plati.csv", inferSchema=True, header=True)

In [4]:
pacienti.createOrReplaceTempView('pacienti')
medici.createOrReplaceTempView('medici')
programari.createOrReplaceTempView('programari')
consultatii.createOrReplaceTempView('consultatii')
istoricMedical.createOrReplaceTempView('istoricMedical')
tratamente.createOrReplaceTempView('tratamente')
facturi.createOrReplaceTempView('facturi')
plati.createOrReplaceTempView('plati')

In [5]:
import pyspark
from pyspark.sql.functions import concat, date_format, hour, coalesce, sum, col

In [6]:
#Vizualizare tabela pacienti
pacienti.show(truncate = False)

+----------+----------+----------+-------------+-------------+--------------------------------------+---------+
|id_pacient|nume      |prenume   |cnp          |data_nasterii|adresa                                |telefon  |
+----------+----------+----------+-------------+-------------+--------------------------------------+---------+
|1         |Popescu   |Ana       |2901505228010|1990-05-15   |Str. Florilor nr.10, Iasi             |721232387|
|2         |Georgescu |Mihai     |2880318229754|1988-03-18   |Str. Mihai Viteazul nr.30, Iasi       |787455632|
|3         |Ionescu   |Ion       |1920608229800|1992-06-08   |Str. Musatini nr.25, Iasi             |721232388|
|4         |Unguranu  |Maria     |6090712228173|2009-07-12   |Str. Tabacului nr.13, Iasi            |787455639|
|5         |Popa      |Constantin|1234567890123|1990-05-15   |Str. Primaverii nr. 10, Bucuresti     |721123456|
|6         |Ionescu   |Vasile    |2345678901234|1985-10-20   |Str. Mihai Eminescu nr. 5, Cluj-Napoca|732

## __Interogarile de la 1-4 sunt realizate de Bargan Diana Georgiana__

### 1. Afisati toate programarile ce au avut loc intre orele 10:00 si 13:30 pe data de 22.03.2024 (selectand numele pacientilor, medicul la care au programare, ora si tipul de programare.)

In [7]:
query1 = spark.sql("""Select concat(nume, ' ', prenume) AS nume_pacient, concat(nume_medic, ' ', prenume_medic) AS nume_medic,\
                    date_format(programari.ora_programare, 'HH:mm') AS ora_programare, tip_tratament
             from programari \
             inner join pacienti ON programari.id_pacient = pacienti.id_pacient \
             inner join medici ON programari.id_medic = medici.id_medic \
             where date(programari.data_programare) == date('2024-03-22') AND
                     hour(programari.ora_programare) BETWEEN 10 AND 13 """)
query1.show(truncate = False)

+---------------+---------------------+--------------+-----------------------+
|nume_pacient   |nume_medic           |ora_programare|tip_tratament          |
+---------------+---------------------+--------------+-----------------------+
|Popescu Ana    |Morari Alexandru     |11:00         |Extractie dentara      |
|Ionescu Ion    |Arion Andreea-Claudia|12:30         |Tratament canal        |
|Popa Constantin|Axinte Alina         |11:00         |Tratament carie dentara|
|Ionescu Mihai  |Axinte Alina         |13:00         |Consultatie            |
+---------------+---------------------+--------------+-----------------------+



### Salvare rezultat obtinut la interogarea nr.1, in fisierul query1_results.csv, in directorul interogari.

In [8]:
output_directory = "/proiectdm1105/interogari/"
output_filename = "query1_results.csv"

# Scrierea rezultatelor în fișierul CSV în HDFS
query1.write.mode("overwrite").format("csv").save(output_directory + "/" + output_filename, header=True)

### Varianta 1, cu urmatoarea comanda putem vedea in linia de comanda continutul rezultatului din fisierul salvat
hdfs dfs -cat /proiectdm1105/interogari/query1_results.csv/part-00000-94e38ee3-2d3d-4b7b-a392-6b28bcbbbf28-c000.csv
### Varianta 2, ar fi sa citim fisierul csv creat anterior si sa il afisam in spark asa cum se observa in codul urmator.

In [8]:
df_query1 = spark.read.csv("/proiectdm1105/interogari/query1_results.csv/part-00000-94e38ee3-2d3d-4b7b-a392-6b28bcbbbf28-c000.csv", inferSchema=True, header=True)
# Afisare continut.
print(df_query1.show())

+---------------+--------------------+-------------------+--------------------+
|   nume_pacient|          nume_medic|     ora_programare|       tip_tratament|
+---------------+--------------------+-------------------+--------------------+
|    Popescu Ana|    Morari Alexandru|2024-05-16 11:00:00|   Extractie dentara|
|    Ionescu Ion|Arion Andreea-Cla...|2024-05-16 12:30:00|     Tratament canal|
|Popa Constantin|        Axinte Alina|2024-05-16 11:00:00|Tratament carie d...|
|  Ionescu Mihai|        Axinte Alina|2024-05-16 13:00:00|         Consultatie|
+---------------+--------------------+-------------------+--------------------+

None


### 2. Care sunt facturile ce nu au fost incasate integral si ce suma mai trebuie de incasat? 

In [9]:
spark.sql("""Select facturi.id_factura, 
            facturi.total_factura - coalesce((Select sum(plati.suma) from plati where plati.id_factura = facturi.id_factura), 0) AS rest_de_incasat
            from facturi  
            where facturi.total_factura - coalesce(
                          (Select sum(plati.suma)
                           from plati 
                           where plati.id_factura = facturi.id_factura), 0) > 0 """).show(truncate=False)

+----------+---------------+
|id_factura|rest_de_incasat|
+----------+---------------+
|111       |750.0          |
|113       |100.0          |
|121       |1500.0         |
+----------+---------------+



### 3. Care sunt pacientii carora li s-au emis facturi in aceeasi zi  in care a fost emisa factura nr.103 ? 

* #### Solutia 1

In [10]:
spark.sql("""Select concat(nume, ' ', prenume) AS nume_pacient from pacienti \
             inner join facturi ON pacienti.id_pacient = facturi.id_pacient \
             where data_emitere IN ( Select data_emitere 
                                     from facturi
                                     where id_factura = 103
                                   ) AND pacienti.id_pacient != (
                                          Select id_pacient from facturi where id_factura = 103 ) """).show(truncate = False)

+---------------+
|nume_pacient   |
+---------------+
|Popescu Ana    |
|Popa Constantin|
|Ionescu Mihai  |
+---------------+



* #### Solutia 2: cu doua niveluri de subconsultari (adica subconsultari in clauza WHERE operatorul IN)

In [11]:
spark.sql(""" Select id_pacient, concat(nume, ' ', prenume) AS nume_pacient 
              from pacienti 
              where id_pacient IN ( Select id_pacient from facturi 
                                    where data_emitere IN (  
                                                 Select data_emitere from facturi
                                                 where id_factura = 103
                                                 )
                                ) AND id_pacient != (
                                            Select id_pacient from facturi 
                                            where id_factura = 103) """).show(truncate = False)                                                                              

+----------+---------------+
|id_pacient|nume_pacient   |
+----------+---------------+
|1         |Popescu Ana    |
|5         |Popa Constantin|
|10        |Ionescu Mihai  |
+----------+---------------+



### 4. Care este pretul mediu al tratamentelor efectuate de un medic in functie de specializare ?

In [12]:
query4 = spark.sql("""Select concat(m.nume_medic,' ', m.prenume_medic) AS nume_medic, m.specializare, \
                    round(AVG(t.cost), 1) AS pret_mediu_tratament
             from medici m
             inner join tratamente t ON m.specializare = t.specializare
             group by m.nume_medic, m.prenume_medic, m.specializare """)
query4.show(truncate=False)

+---------------------+-------------------------+--------------------+
|nume_medic           |specializare             |pret_mediu_tratament|
+---------------------+-------------------------+--------------------+
|Axinte Alina         |Stomatologie generala    |145.0               |
|Tvigun Leia          |Ortodont                 |1350.0              |
|Morari Alexandru     |Chirurgie dento-alveolara|587.5               |
|Arion Andreea-Claudia|Endodontist              |450.0               |
+---------------------+-------------------------+--------------------+



### Salvare rezultat obtinut la interogarea nr.4, in fisierul query4_results - format fisier Parquet, in directorul interogari.

In [14]:
output_directory = "/proiectdm1105/interogari/"
output_filename = "query4_results"
# Scrierea rezultatelor în fișier de tip Parquet în HDFS
query4.write.mode("overwrite").parquet(output_directory + output_filename)

### Citire fisier- format Parquet cu rezultatul interogarii nr. 4 salvat in HDFS

In [13]:
df_parquet = spark.read.format("parquet").load("/proiectdm1105/interogari/query4_results")
# Afisare continut.
print(df_parquet.show(truncate = False))

+---------------------+-------------------------+--------------------+
|nume_medic           |specializare             |pret_mediu_tratament|
+---------------------+-------------------------+--------------------+
|Axinte Alina         |Stomatologie generala    |145.0               |
|Tvigun Leia          |Ortodont                 |1350.0              |
|Morari Alexandru     |Chirurgie dento-alveolara|587.5               |
|Arion Andreea-Claudia|Endodontist              |450.0               |
+---------------------+-------------------------+--------------------+

None


## __Interogarile de la 5-9 sunt realizate de Irimia Nicoletta__

### 5.	Aflati toate programarile de consultatii(data si ora) efectuate de medicul ,,Axinte Alina".

In [14]:
query5 = spark.sql("""
   SELECT pr.data_programare, pr.ora_programare
FROM programari pr
JOIN medici md ON pr.id_medic = md.id_medic
WHERE md.nume_medic = 'Axinte' AND md.prenume_medic = 'Alina' AND pr.tip_tratament= 'Consultatie';
""")
query5.show(truncate=False)

+---------------+-------------------+
|data_programare|ora_programare     |
+---------------+-------------------+
|2024-03-21     |2024-05-16 11:00:00|
|2024-03-21     |2024-05-16 10:00:00|
|2024-03-17     |2024-05-16 11:00:00|
|2024-03-22     |2024-05-16 13:00:00|
+---------------+-------------------+



### 6.Sa se afiseze top 5 tratamente cu cel mai mare cost și să se specifice numele medicului  care efectuează fiecare dintre aceste tratamente.

In [15]:
query6=spark.sql("""
WITH Top5Tratamente AS (
    SELECT
        t.id_tratament,
        t.descriere,
        t.cost,
        t.specializare,
        ROW_NUMBER() OVER (PARTITION BY NULL ORDER BY t.cost DESC) AS ranking
    FROM
        tratamente t
)
SELECT
    id_tratament, descriere, cost, nume_medic
FROM Top5Tratamente 
JOIN medici on Top5Tratamente.specializare = medici.specializare
WHERE
    ranking <= 5
	order by cost desc """)
query6.show(truncate=False)

+------------+--------------------------------------------------------------------+------+----------+
|id_tratament|descriere                                                           |cost  |nume_medic|
+------------+--------------------------------------------------------------------+------+----------+
|2001        |Instalare aparat dentar fix pe ambele arcade                        |3000.0|Tvigun    |
|2002        |Instalare aparat dentar fix pe o singura arcada                     |1500.0|Tvigun    |
|3004        |Implant dentar                                                      |1500.0|Morari    |
|4001        |Tratament endodontic dinte monoradicular(include obturatia de canal)|850.0 |Arion     |
|2004        |Tratament ortodontic                                                |800.0 |Tvigun    |
+------------+--------------------------------------------------------------------+------+----------+



### Salvare rezultat obtinut la interogarea nr.6, in fisierul query6_results - format fisier ORC, in directorul interogari.

In [17]:
output_directory = "/proiectdm1105/interogari/"
output_file = "query6_results.orc"
# Scrierea rezultatelor în fișier de tip ORC în HDFS
query6.write.mode("overwrite").format("orc").save(output_directory + output_file)

### Citire fisier- format ORC cu rezultatul interogarii nr. 6 salvat in HDFS

In [16]:
df_orc = spark.read.format("orc").load("/proiectdm1105/interogari/query6_results.orc")
# Afisare continut.
print(df_orc.show(truncate=False))

+------------+--------------------------------------------------------------------+------+----------+
|id_tratament|descriere                                                           |cost  |nume_medic|
+------------+--------------------------------------------------------------------+------+----------+
|2001        |Instalare aparat dentar fix pe ambele arcade                        |3000.0|Tvigun    |
|2002        |Instalare aparat dentar fix pe o singura arcada                     |1500.0|Tvigun    |
|3004        |Implant dentar                                                      |1500.0|Morari    |
|4001        |Tratament endodontic dinte monoradicular(include obturatia de canal)|850.0 |Arion     |
|2004        |Tratament ortodontic                                                |800.0 |Tvigun    |
+------------+--------------------------------------------------------------------+------+----------+

None


### 7. Care este numărul total de consultații pentru care diagnosticul contine cuvantul "Carie dentara” .

In [17]:
query7 = spark.sql(""" SELECT
    COUNT(*)AS consultatii_diagnostic_carie_dentara
FROM 
    consultatii
WHERE 
    diagnostic LIKE '%Carie dentara%'; """)

query7.show(truncate=False)

+------------------------------------+
|consultatii_diagnostic_carie_dentara|
+------------------------------------+
|3                                   |
+------------------------------------+



### 8.Sa se calculeze incasarile facturilor saptamanale.

In [18]:
query8 = spark.sql("""
SELECT DATE_TRUNC('week', data_emitere) AS start_of_week, SUM(total_factura) AS incasari_saptamanale
FROM facturi
GROUP BY DATE_TRUNC('week', data_emitere)
ORDER BY start_of_week;
""")
query8.show(truncate=False)

+-------------------+--------------------+
|start_of_week      |incasari_saptamanale|
+-------------------+--------------------+
|2024-03-11 00:00:00|50.0                |
|2024-03-18 00:00:00|4300.0              |
|2024-03-25 00:00:00|6000.0              |
+-------------------+--------------------+



### 9. Afisati toti pacientii care au avut plati intre 200 de lei si 1000 lei în total.

In [19]:
#definire anterioara a query-ului 
query = """
SELECT p.id_pacient, p.nume, p.prenume, SUM(pl.suma) AS suma_totala
FROM pacienti p
    JOIN facturi f ON p.id_pacient = f.id_pacient
    JOIN plati pl ON f.id_factura = pl.id_factura
GROUP BY
    p.id_pacient, p.nume, p.prenume
HAVING
    SUM(pl.suma) BETWEEN 200 AND 1000
ORDER BY
    p.id_pacient
"""
#memorare rezultat 
result = spark.sql(query)
#afisare rezultat
result.show(truncate=False)

+----------+----------+----------+-----------+
|id_pacient|nume      |prenume   |suma_totala|
+----------+----------+----------+-----------+
|1         |Popescu   |Ana       |350.0      |
|2         |Georgescu |Mihai     |200.0      |
|3         |Ionescu   |Ion       |900.0      |
|5         |Popa      |Constantin|200.0      |
|6         |Ionescu   |Vasile    |850.0      |
|7         |Radu      |Maria     |210.0      |
|8         |Dumitrescu|Andrei    |360.0      |
|10        |Ionescu   |Mihai     |200.0      |
+----------+----------+----------+-----------+



## __Interogarile de la 10-13 sunt realizate de Vasilache Vlad__

### 10. Extrageți ora programata la Consultatie pentru un anumit pacient, pe baza CNP-ului: 6090712228173.

In [24]:
query10 = spark.sql("""
SELECT date_format(p.ora_programare,'HH:mm') AS ora_programare
FROM programari p
JOIN pacienti pac ON p.id_pacient = pac.id_pacient
WHERE pac.CNP = '6090712228173' AND p.tip_tratament = 'Consultatie'
""")
query10.show(truncate=False)

+--------------+
|ora_programare|
+--------------+
|11:30         |
+--------------+



### 11. Extrageti toate programările pentru pacientul cu CNP-ul 2704091234568

In [27]:
query11 = spark.sql("""
SELECT id_programare, data_programare, 
       date_format(ora_programare,'HH:mm') AS ora_programare, tip_tratament
FROM programari 
WHERE id_pacient = (SELECT id_pacient 
					FROM pacienti
					WHERE CNP = '6090712228173')
""")
query11.show(truncate=False)

+-------------+---------------+--------------+-----------------------+
|id_programare|data_programare|ora_programare|tip_tratament          |
+-------------+---------------+--------------+-----------------------+
|4            |2024-03-21     |11:30         |Consultatie            |
|8            |2024-03-25     |11:00         |Instalare aparat dentar|
+-------------+---------------+--------------+-----------------------+



### 12. Afișează lista cu programarile între orele 11:00 si 12:30 pentru toate zilele.

In [29]:
query12 = spark.sql("""
SELECT CONCAT(pac.nume, ' ', pac.prenume) AS nume_pacient,
       CONCAT(med.nume_medic, ' ', med.prenume_medic) AS nume_medic,
       p.data_programare,
       date_format(p.ora_programare,'HH:mm') AS ora_programare,
       p.tip_tratament
FROM programari p
INNER JOIN pacienti pac ON p.id_pacient = pac.id_pacient
INNER JOIN medici med ON p.id_medic = med.id_medic
WHERE p.ora_programare BETWEEN '11:00' AND '12:30';
""")
query12.show(truncate=False)

+---------------+---------------------+---------------+--------------+-----------------------+
|nume_pacient   |nume_medic           |data_programare|ora_programare|tip_tratament          |
+---------------+---------------------+---------------+--------------+-----------------------+
|Georgescu Mihai|Axinte Alina         |2024-03-21     |11:00         |Consultatie            |
|Unguranu Maria |Tvigun Leia          |2024-03-21     |11:30         |Consultatie            |
|Popescu Ana    |Morari Alexandru     |2024-03-22     |11:00         |Extractie dentara      |
|Ionescu Ion    |Arion Andreea-Claudia|2024-03-22     |12:30         |Tratament canal        |
|Unguranu Maria |Tvigun Leia          |2024-03-25     |11:00         |Instalare aparat dentar|
|Popa Constantin|Axinte Alina         |2024-03-22     |11:00         |Tratament carie dentara|
|Georgescu Elena|Axinte Alina         |2024-03-17     |11:00         |Consultatie            |
|Georgescu Elena|Axinte Alina         |2024-03-18 

### 13.Care sunt zilele in care s-au intocmit (emis) cel puțin 3 facturi ?

In [13]:
query13 = spark.sql("""
SELECT data_emitere, COUNT(*) AS numar_facturi
FROM facturi
GROUP BY data_emitere
HAVING COUNT(*) >= 3;
""")
query13.show(truncate=False)

+------------+-------------+
|data_emitere|numar_facturi|
+------------+-------------+
|2024-03-23  |3            |
|2024-03-21  |6            |
|2024-03-22  |4            |
|2024-03-25  |3            |
+------------+-------------+



### Salvarea rezultatului obtinut la interogarea nr.12, in fisier format fisier ORC, in directorul interogari.

In [30]:
output_directory = "/proiectdm1105/interogari/"
output_file = "query12_results.orc"
# Scrierea rezultatelor în fișier de tip ORC în HDFS
query12.write.mode("overwrite").format("orc").save(output_directory + output_file)

### Citire fisier format ORC cu rezultatul interogarii nr. 12 salvat in HDFS

In [31]:
df_orc12 = spark.read.format("orc").load("/proiectdm1105/interogari/query12_results.orc")
# Afisare continut.
print(df_orc12.show(truncate=False))

+---------------+---------------------+---------------+--------------+-----------------------+
|nume_pacient   |nume_medic           |data_programare|ora_programare|tip_tratament          |
+---------------+---------------------+---------------+--------------+-----------------------+
|Georgescu Mihai|Axinte Alina         |2024-03-21     |11:00         |Consultatie            |
|Unguranu Maria |Tvigun Leia          |2024-03-21     |11:30         |Consultatie            |
|Popescu Ana    |Morari Alexandru     |2024-03-22     |11:00         |Extractie dentara      |
|Ionescu Ion    |Arion Andreea-Claudia|2024-03-22     |12:30         |Tratament canal        |
|Unguranu Maria |Tvigun Leia          |2024-03-25     |11:00         |Instalare aparat dentar|
|Popa Constantin|Axinte Alina         |2024-03-22     |11:00         |Tratament carie dentara|
|Georgescu Elena|Axinte Alina         |2024-03-17     |11:00         |Consultatie            |
|Georgescu Elena|Axinte Alina         |2024-03-18 